1. 문서의 내용을 읽기
2. 문서 쪼개기
  - 토큰 수 초과로 답변을 생성하지 못할 수 있고
  - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, Vector DB에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 전달
6. 답변 생성

# 1. 문서의 내용을 읽기([🔗link](https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/))


`--upgrade`: 현재 package가 설치되어있다면 구 버전을 제거하고 최신 버전으로 다시 설치하는 옵션._**(shortcut: -U)**_  
`--quiet`: 설치 과정에서 출력되는 메시지를 최소화하는 옵션._**(shortcut: -q)**_  

```소득세법을 다운 받으면 `doc`파일형식인데 mac에서 변환하는 방법이 없어 web에서 `doc to docx`로 파일을 변환하여 다운받아서 진행함```

In [2]:
# %pip install --upgrade --quiet  docx2txt

# %pip install -Uq docx2txt langchain_community langchain-text-splitters langchain-openai langchain-chroma python-dotenv langchain

In [3]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax.docx")
data = loader.load()

# 2. 문서 쪼개기([🔗link](https://python.langchain.com/docs/tutorials/rag/#splitting-documents))
`langchain_text_splitters` 패키지를 사용하여 문서를 더 작은 `Document` 객체로 쪼갠다.

- `chunk_size`: 쪼갤 문서의 크기 => 하나의 chunk에서 몇 개의 토큰을 가지는지 지정
- `chunk_overlap`: 쪼갠 문서의 중복되는 부분의 크기로 _**앞 뒤 context를 추가하기 위해 사용**_

```python
Document 객체는 다음과 같은 속성을 가진다
{
  "page_content": "문서의 실제 텍스트 내용",
  "metadata": {
    "source": "문서의 출처/경로",
    "start_index": "원본 문서에서의 시작 위치 (add_start_index=True 설정 시)"
  }
}
```


## 2-1. 문서 쪼개기 예시

`RecursiveCharacterTextSplitter` 클래스를 사용해서 문서를 쪼개며 쪼갠 문서는 `List[Document]` 형태로 반환된다.  

아래에서는 `split_documents` 메서드를 사용하지만 _**load와 통합하기 위해 text_splitter를 인자로 넘겨주어 뒷단에서 처리**_하도록 한다.

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200,
  add_start_index=True,
)

all_splits = text_splitter.split_documents(data)

# print(f"sub documents: {len(all_splits)}")
# print(all_splits[0].page_content)
# print(all_splits[0].metadata)


## 2-2. loader와 text_splitter 통합

In [5]:
document_list = loader.load_and_split(text_splitter=text_splitter) # loader와 text_splitter를 통합
first_document = document_list[0]

print(first_document.page_content)
print(first_document.metadata)

소득세법

소득세법

[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]

기획재정부(재산세제과(양도소득세)) 044-215-4312

기획재정부(소득세제과(근로소득)) 044-215-4216

기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233

기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217



제1장 총칙 <개정 2009. 12. 31.>



제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.

[본조신설 2009. 12. 31.]

[종전 제1조는 제2조로 이동 <2009. 12. 31.>]



제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>

1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.

2. “비거주자”란 거주자가 아닌 개인을 말한다.

3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.

4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.

5. “사업자”란 사업소득이 있는 거주자를 말한다.

② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.

[본조신설 2009. 12. 31.]



제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인

② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자

3. 내국법인

4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다.

# 3. 임베딩 & 벡터화 데이터 저장

- `한글`을 임베딩할때는 강사가 추천해준 `upstage`의 `solar model`이 가장 좋다고 한다.([🔗upstage](https://python.langchain.com/docs/integrations/text_embedding/upstage/))  
- 아래 연습 코드에서는 `text-embedding-3-large`모델을 가지고 진행한다.(text-embedding-3-small도 있지만 성능이 떨어짐.)


## 3.1 벡터화([🔗embedding](https://python.langchain.com/docs/integrations/text_embedding/))
위에서 문서를 더 작은 Document 형식으로 text_splitter를 통해 나누었으니 이제 chroma에 넣기 전에 document를 임베딩하여 `vector화`를 진행해야 한다.

In [6]:
# dotenv에 openai api key가 존재하므로 langchain의 document에서의 api key를 가져올 필요가 없다. 
# 기본적으로 "OPENAI_API_KEY"로 설정되어 있어 뒷단에서 자동으로 가져온다.

from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

embedding.embed_query("hello") # 임베딩 test

[-0.024626221507787704,
 -0.007549732457846403,
 0.0039802188985049725,
 0.006015626713633537,
 0.005089526530355215,
 0.0002501478011254221,
 -0.006442438345402479,
 0.05244144797325134,
 0.00959117989987135,
 0.056532397866249084,
 -0.004312407225370407,
 -0.02127615176141262,
 -0.021582167595624924,
 -0.010380378924310207,
 -0.017893873155117035,
 0.02014872618019581,
 0.02415914461016655,
 0.005955229047685862,
 0.015099464915692806,
 -0.011910458095371723,
 0.010605864226818085,
 -0.00815773755311966,
 0.0010781018063426018,
 0.03504686430096626,
 0.010034097358584404,
 0.031245825812220573,
 -0.020841287449002266,
 -0.01724962890148163,
 -0.04928465187549591,
 -0.03375837579369545,
 0.0008209075895138085,
 0.026349572464823723,
 0.0019427977968007326,
 0.019053511321544647,
 0.03565889596939087,
 0.007155132945626974,
 0.026655588299036026,
 0.004376831464469433,
 -0.023869233205914497,
 0.022403579205274582,
 0.026655588299036026,
 0.00867313239723444,
 -0.024449054151773453,
 -

## 3.2 chroma에 vector화된 document 저장([🔗VerctorDB-Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/))

Chroma는 `in-memory db`이므로 작업환경이 달라지면 데이터가 사라진다. 따라서 데이터를 영구적으로 저장하기 위해서는 `persist_directory`를 지정해줘야 한다. `persist_directory`를 이용해서 `벡터화된 document, metadata, indexing`이 저장되어 추후 빠르게 불러서 사용할 수 있다.
  
from_documents args:
1. documents: 벡터화된 document
2. embedding: 임베딩 모델
3. collection_name: 데이터베이스 컬렉션 이름
4. persist_directory: 데이터베이스 저장 경로


In [ ]:
from langchain_chroma import Chroma

tax_db_path = "./tax_db"
tax_collection_name = "tax"
# 데이터를 처음 저장할 때 
# Chroma.from_documents(documents=document_list, embedding=embedding, collection_name=tax_collection_name, persist_directory=tax_db_path)

# 이미 vector화 된 db를 chroma로 불러오기
db = Chroma(
  persist_directory=tax_db_path, 
  embedding_function=embedding, 
  collection_name=tax_collection_name
)


# 4. Retrieval(검색)
Chroma를 통해서 vector화된 Document를 저장했으니 사용자의 질문과 유사한 문서를 검색해서 가져오는 작업을 해야한다.  

retrieval은 질문(`query`)와 유사한 문서를 검색한다.  

`질문(query) vector화 -> 유사도 검색을 통한 문서 검색`  

대부분의 vector database는 `similarity_search` 메서드를 통해서 query(질문)를 알아서 <span style="color:orange">vector화하고 유사도 검색을 통해 문서를 검색</span>한다.

유사도 중 `코사인 유사도(cosine similarity)`에 대해서 알아보자.
=> 두 개 의 `단어 사이의 각도`를 통해서 단어가 얼마나 비슷한지에 대한 수치를 나타낸다.

```python
import numpy as np

def cosine_similarity(vec1, vec2):
  """
  Calculate the cosine similarity between two vectors.

  Parameters:
  vec1 (numpy array): First vector
  vec2 (numpy array): second vector
  
  Returns:
  float: Cosine similarity between vec1 and vec2
  """
  dot_product = np.dot(vec1, vec2)
  norm_vec1 = np.linalg.norm(vec1)
  norm_vec2 = np.linalg.norm(vec2)

  if norm_vec1 == 0 or norm_vec2 == 0:
    return 0.0
  
  return dot_product / (norm_vec1 * norm_vec2)

king_embedded_response = client.embeddings.create(input="king", model="text-embedding-3-large")
queen_embedded_response = client.embeddings.create(input="queen", model="text-embedding-3-large")

king_vector = np.array(king_embedded_response.data[0].embedding)
queen_vector = np.array(queen_embedded_response.data[0].embedding)

king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print(king_queen_similarity)
```




In [8]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?" # 존댓말로 하면 더 좋게 나온다는 썰이 있음

retrieved_docs = db.similarity_search(query=query, k=3) # k는 검색할 문서의 개수

retrieved_docs

[Document(id='3b79e50b-4c94-4120-893d-abfe938c7ae0', metadata={'source': './tax.docx', 'start_index': 11798}, page_content='바. 「문화유산의 보존 및 활용에 관한 법률」에 따라 국가지정문화유산으로 지정된 서화ㆍ골동품의 양도로 발생하는 소득\n\n사. 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로써 발생하는 소득\n\n아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득\n\n\u3000\u3000\u3000\u30001) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금\n\n\u3000\u3000\u3000\u30002) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대\n\n\u3000\u3000\u3000\u30003) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액\n\n\u3000\u3000\u3000\u30004) 종교관련종사자 또는 그 배우자의 출산이나 6세 이하(해당 과세기간 개시일을 기준으로 판단한다) 자녀의 보육과 관련하여 종교단체로부터 받는 금액으로서 월 20만원 이내의 금액\n\n\u3000\u3000\u3000\u30005) 종교관련종사자가 기획재정부령으로 정하는 사택을 제공받아 얻는 이익\n\n자. 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당\n\n[전문개정 2009. 12. 31.]\n\n\n\n제13조 삭제 <2009. 12. 31.>\n\n\n\n제2절 과세표준과 세액의 계산 <개정 2009. 12. 31.>\n\n\n\n제1관 세액계산 통칙 <개정 2009. 12. 31.>\n\n\n\n제14조(과세표준의 계산) ① 거주자의 종합소득 및 퇴직소득에 대한 과세표준은 각각 구분하여 계산한다.\n\n② 종합소득에 대한 

# 5. Augmentation을 위한 Prompt 활용

위에서 retrieval된 data를 llm이 `이해하고, 원하는 방식으로 동작하도록 설계된 입력테스트`를 prompt에 담아서 llm에 전달하여 답변을 얻을 수 있다.

In [ ]:
prompt = f"""
[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

'연봉 5000만원인 직장인의 소득세를 계산하기 위해서는 다음과 같은 단계가 필요합니다. 워낙 여러 요인이 계산에 영향을 미치기에 간단한 예시로 설명드리겠습니다.\n\n1. **과세표준 계산**: 연봉 5000만원에서 소득공제를 차감하여 과세표준을 계산해야 합니다. 소득공제에는 다양한 항목이 포함될 수 있으며 각 개인의 상황에 따라 다르므로 구체적인 금액을 알 수 없습니다. 보통 기본 공제와 여러 다른 공제를 합산하여 차감합니다.\n\n2. **세율 적용**: 한국의 소득세는 누진세율 구조를 가지고 있어 과세표준에 해당되는 구간에 따라 세율이 다르게 적용됩니다. (예를 들어, 1200만원 이하는 6%, 1200만원 초과~4600만원 이하는 15%, 4600만원 초과~8800만원 이하는 24% 등의 세율이 있습니다.)\n\n3. **지방소득세**: 소득세의 10%에 해당하는 금액이 지방소득세로 추가됩니다.\n\n실제 소득세 금액은 개인의 소득 공제 항목과 규모에 따라 달라지므로, 정확한 계산을 위해서는 개인의 구체적인 소득구성, 공제 사항 등을 반영한 시뮬레이션 혹은 국세청의 간이세액표를 참조하는 것이 필요합니다. 일반적으로는 회사에서 연말정산 시 모든 공제 항목을 고려하여 정확한 계산을 수행합니다.'

# 6. 질문과 생성

llm에게 <span style="color:orange">사용자의 질이의 유사도 검색을 통해 얻은 context</span>를 prompt에 담았으니  
prompt를 llm에 전달하여 질문에 대한 `답변`을 생성하자.

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
result = llm.invoke(prompt)

result.content


'한국 소득세 계산은 여러 요소를 고려하여 이루어지며, 연봉 5000만 원인 직장인의 소득세를 계산하기 위해서는 기본적인 세율과 공제 항목을 알아야 합니다. 일반적인 경우를 가정하여 소득세를 추정해 보겠습니다.\n\n1. **과세표준 계산**: 연봉에서 각종 소득 공제를 차감하여 과세표준을 계산합니다. 예를 들어, 근로소득공제, 국민연금, 건강보험료 등이 있습니다. 연봉 5000만 원에 대한 대략적인 근로소득공제는 다음과 같습니다:\n\n   - 근로소득공제는 연 3600만원 이하일 때 기본적으로 70%입니다. 5000만 원의 경우, 공제액은 약 1100만 원이 될 것입니다.\n\n2. **과세표준에 대한 세율**: 대한민국의 소득세율은 누진세율 구조로 되어 있으며, 다음과 같은 세율이 적용됩니다. 대략적인 계산을 위해 아래의 세율을 참고하십시오(최근 법령 기준임을 가정합니다).\n\n   - 1200만 원 이하: 6%\n   - 1200만 원 초과 4600만 원 이하: 15%\n   - 4600만 원 초과 8800만 원 이하: 24%\n   - ...\n\n   예를 들어, 근로소득공제 후 과세표준이 3900만 원(5000만 원 - 1100만 원 공제)이라면, 이 과세표준에 대해 위의 세율을 적용합니다.\n\n3. **세액계산**: 공제 후 과세표준에 해당 세율을 적용하여 산출세액을 계산합니다.\n\n4. **세액 공제 및 감면**: 최종 세액에서 다양한 세액공제(예: 근로소득세액 공제, 자녀세액공제 등)를 적용하여 최종 납부할 세액을 계산합니다.\n\n대략적인 설명이므로, 정밀한 계산을 위해서는 더 구체적인 정보와 최신 세법 정보 및 공제 항목을 적용해야 합니다. 필요하다면, 국세청의 세금계산기를 활용하거나 세무 전문가에게 상담 받는 것이 좋습니다.'

# 7. langchain을 활용한 자동화 과정(QA_chain)

위에서 prompt에 제공할 context인 data를 가져오는 `Retrieval`, `Augmentation`를 위해 prompt를 제공, 이후 llm.invoke를 통한 질문 생성`Generation`까지 일일히 작업을 하였다.  

하지만, `langchain`을 활용하면 위의 일련의 과정을 자동화할 수 있다.

In [15]:
from langchain import hub
from langchain.chains import RetrievalQA

prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  retriever=db.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"
result = qa_chain({"query": query})

result

/Users/pgj/.pyenv/versions/3.13.3/envs/inflearn-rag-llm-application/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/var/folders/3y/cwcdqhln277bg0tvwt9gp2dr0000gn/T/ipykernel_17752/1523509034.py:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '해당 문맥에서는 연봉 5,000만원인 직장인의 소득세에 관한 직접적인 정보가 제공되지 않았습니다. 소득세 계산 기준이나 비과세 소득, 공제 항목 등에 대한 전반적인 규정은 있지만, 특정 연봉에 대한 소득세 액수는 구체적으로 제시되어 있지 않습니다. 소득세 계산은 총소득에서 공제 항목을 제외한 과세표준을 바탕으로, 차등 세율을 적용하여 산출되므로, 자세한 계산은 세법 규정과 개인의 상황에 따라 다를 수 있습니다.'}